In [1]:
from scaaml.io import Dataset
import numpy as np

# specify dummy data

In [2]:
root_path = './'
architecture = 'arch'
implementation = 'implem'
algorithm = 'algo'
version = 1
minfo = {
    "trace1": {
        "type": "power",
        "len": 1024,
    }
}

apinfo = {
    "key": {
        "len": 16,
        "max_val": 256
    }
}
chip_id = 1   # which chip this was captured on
comment= "this is a test" 
purpose = "train"
example_per_shard = 1

## generate fake data

In [3]:
key = np.random.randint(0, 255, 16)
key2 = np.random.randint(0, 255, 16)
trace1 = np.random.rand(1024)

# Creating dataset

In [4]:
## init

In [5]:
ds = Dataset(root_path=root_path,
             architecture=architecture,
             implementation=implementation,
             algorithm=algorithm,
             version=version,
             purpose=purpose,
             comment=comment,
             chip_id=chip_id,
             examples_per_shard=example_per_shard,
             measurements_info=minfo,
             attack_points_info=apinfo)

[Warning] Path exist, some files might be over-written
Dataset path: algo_arch_implem_v1_train


## writing shard

In [6]:
ds.new_shard(key, 1, split='train')
ds.write_example({"key": key,
                  #"sub_byte_in": key
                  }, {"trace1": trace1})
ds.close_shard()

train
defaultdict(<class 'int'>, {})


## Using a dataset

In [9]:
dataset_path = './algo_arch_implem_v1_train'

# display dataset info

In [10]:

Dataset.summary(dataset_path)

[Dataset Summary]
Info
--------------  --------------
architecture    arch
implementation  implem
algorithm       algo
version         1
chip_id         1
comment         this is a test
purpose         train
compression     GZIP
--------------  --------------

Attack Points
ap      len    max_val
----  -----  ---------
key      16        256

Measurements
name    type      len
------  ------  -----
trace1  power    1024

Content
split      num_keys    num_examples
-------  ----------  --------------
train             1               1


In [11]:
trace_len = 1024

train_ds, inputs, outputs = Dataset.as_tfdataset(dataset_path, 
                                           split='train', 
                                           attack_points='key',
                                           traces='trace1',
                                           traces_max_len=trace_len,
                                           bytes=1,
                                           shards=1)

reloading algo_arch_implem_v1_train\info.json
Dataset path: algo_arch_implem_v1_train\algo_arch_implem_v1_train


In [20]:
for batch in train_ds.take(1):
    print('x ->', batch[0].keys(), batch[0]['trace1'].shape)
    print('y ->', batch[1].keys(), batch[1]['key_1'].shape)

x -> dict_keys(['trace1']) (32, 32, 32)
y -> dict_keys(['key_1']) (32, 256)
